---
## 1. Teoretický Úvod

### 1.1 Proč Random Forest pro Imputaci?

**Random Forest** je ensemble metoda kombinující rozhodovací stromy:

$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B} T_b(x)$$

kde:
- $B$ = počet stromů (n_estimators)
- $T_b(x)$ = predikce b-tého stromu

#### Výhody pro imputaci:

| Vlastnost | Proč je důležitá |
|-----------|------------------|
| **Nelineární vztahy** | Zachycuje komplexní závislosti mezi OHLCV a fundamenty |
| **Robustnost vůči outlierům** | Finanční data obsahují extrémy (NVDA +200%) |
| **Multi-output podpora** | Predikuje všech 14 metrik najednou |
| **Feature importance** | Interpretovatelnost - které features jsou klíčové |
| **Žádná normalizace** | Stromy nepotřebují škálování dat |

### 1.2 Multi-Output Regrese

Místo 14 samostatných modelů trénujeme jeden **MultiOutputRegressor**:

$$f: \mathbb{R}^{18} \rightarrow \mathbb{R}^{14}$$

**Vstup (18 features):**
- OHLCV: open, high, low, close, volume
- Technické: returns, volatility, rsi, macd, sma, ema, ...

**Výstup (14 targets):**
- Valuace: PE, PB, PS, EV_EBITDA
- Profitabilita: ROE, ROA, marže (3)
- Zdraví: Debt/Equity, Current/Quick Ratio (3)
- Růst: Revenue/Earnings Growth (2)

---
## 2. Setup Prostředí

In [ ]:
# Instalace (pro Colab)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
import joblib

# Scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')
np.random.seed(42)

print("✓ Knihovny načteny")

In [ ]:
# Připojení Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    print("ℹ️ Lokální prostředí")

# Cesty
DATA_PATH = f"{DRIVE_PATH}/data"
MODEL_PATH = f"{DRIVE_PATH}/models"
os.makedirs(MODEL_PATH, exist_ok=True)

---
## 3. Načtení Dat

In [ ]:
# Načtení OHLCV dat z Notebooku 01
ohlcv_path = f"{DATA_PATH}/ohlcv/all_sectors_ohlcv_10y.csv"
ohlcv_df = pd.read_csv(ohlcv_path, parse_dates=['date'])

print(f"📈 OHLCV Data:")
print(f"   Záznamů: {len(ohlcv_df):,}")
print(f"   Tickerů: {ohlcv_df['ticker'].nunique()}")
print(f"   Období: {ohlcv_df['date'].min().strftime('%Y-%m')} → {ohlcv_df['date'].max().strftime('%Y-%m')}")

In [ ]:
# Načtení fundamentálních dat
fund_path = f"{DATA_PATH}/fundamentals/all_sectors_fundamentals.csv"
fundamentals_df = pd.read_csv(fund_path, parse_dates=['date'])

print(f"\n📊 Fundamentální Data:")
print(f"   Záznamů: {len(fundamentals_df)}")
print(f"   Tickerů: {fundamentals_df['ticker'].nunique()}")

# Zobrazení prvních záznamů
display(fundamentals_df.head())

---
## 4. Příprava Trénovacích Dat

### 4.1 Strategie

1. **Merge** OHLCV dat s fundamenty podle tickeru
2. **Forward-fill** fundamentů pro vytvoření většího datasetu
3. **Výběr features** (OHLCV + technické) a **targets** (14 fundamentů)
4. **Odstranění NaN** hodnot

In [ ]:
# Definice features a targets

# OHLCV + technické indikátory (features)
FEATURE_COLS = [
    'open', 'high', 'low', 'close', 'volume',
    'returns', 'volatility_12m', 'rsi_14',
    'macd', 'macd_signal', 'macd_hist',
    'sma_3', 'sma_6', 'sma_12',
    'ema_3', 'ema_6', 'ema_12',
    'volume_change', 'price_momentum'
]

# Fundamentální metriky (targets)
TARGET_COLS = [
    'PE', 'PB', 'PS', 'EV_EBITDA',
    'ROE', 'ROA', 'Profit_Margin', 'Operating_Margin', 'Gross_Margin',
    'Debt_to_Equity', 'Current_Ratio', 'Quick_Ratio',
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'
]

print(f"📊 Features: {len(FEATURE_COLS)}")
for f in FEATURE_COLS:
    print(f"   • {f}")

print(f"\n🎯 Targets: {len(TARGET_COLS)}")
for t in TARGET_COLS:
    print(f"   • {t}")

In [ ]:
def prepare_training_data(ohlcv: pd.DataFrame, fundamentals: pd.DataFrame) -> pd.DataFrame:
    """
    Připraví trénovací data spojením OHLCV s fundamenty.
    
    Strategie:
    1. Pro každý ticker přidáme fundamenty k nejnovějším OHLCV datům
    2. Používáme forward-fill pro rozšíření datasetu
    """
    merged_data = []
    
    for ticker in ohlcv['ticker'].unique():
        # OHLCV pro tento ticker
        ticker_ohlcv = ohlcv[ohlcv['ticker'] == ticker].copy()
        ticker_ohlcv = ticker_ohlcv.sort_values('date')
        
        # Fundamenty pro tento ticker
        ticker_fund = fundamentals[fundamentals['ticker'] == ticker]
        
        if ticker_fund.empty:
            continue
        
        # Přidáme fundamenty ke všem OHLCV záznamům (forward fill simulace)
        for col in TARGET_COLS:
            if col in ticker_fund.columns:
                ticker_ohlcv[col] = ticker_fund[col].values[0]
            else:
                ticker_ohlcv[col] = np.nan
        
        merged_data.append(ticker_ohlcv)
    
    result = pd.concat(merged_data, ignore_index=True)
    return result

# Příprava dat
print("🔄 Příprava trénovacích dat...")
training_df = prepare_training_data(ohlcv_df, fundamentals_df)

print(f"\n✓ Merged dataset: {len(training_df):,} záznamů")

In [ ]:
# Odstranění řádků s NaN v features nebo targets
print("🧹 Čištění dat...")

# Před čištěním
print(f"   Před čištěním: {len(training_df):,} záznamů")

# Kontrola dostupných sloupců
available_features = [f for f in FEATURE_COLS if f in training_df.columns]
available_targets = [t for t in TARGET_COLS if t in training_df.columns]

print(f"   Dostupné features: {len(available_features)}/{len(FEATURE_COLS)}")
print(f"   Dostupné targets: {len(available_targets)}/{len(TARGET_COLS)}")

# Odstranění NaN
all_cols = available_features + available_targets
clean_df = training_df.dropna(subset=all_cols)

print(f"   Po čištění: {len(clean_df):,} záznamů")
print(f"   Odstraněno: {len(training_df) - len(clean_df):,} záznamů")

In [ ]:
# Příprava X (features) a y (targets)
X = clean_df[available_features].values
y = clean_df[available_targets].values

print(f"\n📊 Finální dataset:")
print(f"   X shape: {X.shape} (samples × features)")
print(f"   y shape: {y.shape} (samples × targets)")

---
## 5. Train/Test Split

### 5.1 Chronologický Split

Pro finanční data **NIKDY** nepoužíváme náhodný split (data leakage).

Používáme **chronologický split**:
- **Training**: 2015-2023 (80%)
- **Test**: 2024-2025 (20%)

In [ ]:
# Chronologický split
# Protože máme forward-filled data, použijeme jednoduchý split podle indexu

split_idx = int(len(X) * 0.8)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"📊 Train/Test Split:")
print(f"   Training: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test: {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# Standardizace features (pro konzistenci, i když RF ji nepotřebuje)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardizovány")

---
## 6. Trénování Random Forest Modelu

### 6.1 Hyperparametry

| Parametr | Hodnota | Vysvětlení |
|----------|---------|------------|
| `n_estimators` | 200 | Počet stromů - více = stabilnější |
| `max_depth` | 15 | Hloubka stromu - prevence overfittingu |
| `min_samples_split` | 5 | Min. samples pro split |
| `min_samples_leaf` | 2 | Min. samples v listu |
| `n_jobs` | -1 | Paralelizace na všech CPU |

In [ ]:
# Konfigurace modelu
RF_PARAMS = {
    'n_estimators': 200,
    'max_depth': 15,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'n_jobs': -1
}

print("🌲 Random Forest Konfigurace:")
for param, value in RF_PARAMS.items():
    print(f"   {param}: {value}")

In [ ]:
%%time

# Trénování modelu
print("🚀 Trénování Multi-Output Random Forest...")
print(f"   Targets: {len(available_targets)}")
print(f"   Training samples: {len(X_train):,}")
print()

# Vytvoření a trénování modelu
base_rf = RandomForestRegressor(**RF_PARAMS)
model = MultiOutputRegressor(base_rf)

model.fit(X_train_scaled, y_train)

print("\n✅ Model natrénován!")

---
## 7. Evaluace Modelu

### 7.1 Metriky

Pro každou fundamentální metriku počítáme:

| Metrika | Formule | Interpretace |
|---------|---------|-------------|
| **MAE** | $\frac{1}{n}\sum|y_i - \hat{y}_i|$ | Průměrná absolutní chyba |
| **RMSE** | $\sqrt{\frac{1}{n}\sum(y_i - \hat{y}_i)^2}$ | Penalizuje velké chyby |
| **R²** | $1 - \frac{SS_{res}}{SS_{tot}}$ | Vysvětlená variance (0-1) |

In [ ]:
# Predikce na test setu
y_pred = model.predict(X_test_scaled)

print(f"✓ Predikce dokončeny: {y_pred.shape}")

In [ ]:
# Evaluace pro každý target
results = []

print("📊 EVALUACE MODELU")
print("="*70)
print(f"{'Target':<25} {'MAE':>10} {'RMSE':>10} {'R²':>10}")
print("-"*70)

for i, target in enumerate(available_targets):
    y_true_i = y_test[:, i]
    y_pred_i = y_pred[:, i]
    
    mae = mean_absolute_error(y_true_i, y_pred_i)
    rmse = np.sqrt(mean_squared_error(y_true_i, y_pred_i))
    r2 = r2_score(y_true_i, y_pred_i)
    
    results.append({
        'Target': target,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })
    
    # Barevné označení R²
    r2_color = '🟢' if r2 > 0.5 else '🟡' if r2 > 0.2 else '🔴'
    print(f"{target:<25} {mae:>10.3f} {rmse:>10.3f} {r2:>9.3f} {r2_color}")

print("-"*70)

# Průměrné metriky
results_df = pd.DataFrame(results)
print(f"{'PRŮMĚR':<25} {results_df['MAE'].mean():>10.3f} {results_df['RMSE'].mean():>10.3f} {results_df['R2'].mean():>9.3f}")

print("\n🟢 R² > 0.5: Dobrá predikce")
print("🟡 R² 0.2-0.5: Střední predikce")
print("🔴 R² < 0.2: Slabá predikce")

In [ ]:
# Vizualizace výsledků
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. R² pro každý target
ax1 = axes[0, 0]
colors = ['green' if r > 0.5 else 'orange' if r > 0.2 else 'red' 
          for r in results_df['R2']]
bars = ax1.barh(results_df['Target'], results_df['R2'], color=colors)
ax1.axvline(0.5, color='green', linestyle='--', alpha=0.5, label='Dobrá (0.5)')
ax1.axvline(0.2, color='orange', linestyle='--', alpha=0.5, label='Střední (0.2)')
ax1.set_xlabel('R² Score')
ax1.set_title('R² Score pro Každou Fundamentální Metriku', fontweight='bold')
ax1.legend(loc='lower right')
ax1.set_xlim(-0.1, 1.0)

# 2. MAE vs RMSE
ax2 = axes[0, 1]
x = np.arange(len(results_df))
width = 0.35
ax2.bar(x - width/2, results_df['MAE'], width, label='MAE', color='steelblue')
ax2.bar(x + width/2, results_df['RMSE'], width, label='RMSE', color='coral')
ax2.set_xticks(x)
ax2.set_xticklabels(results_df['Target'], rotation=45, ha='right')
ax2.set_ylabel('Chyba')
ax2.set_title('MAE vs RMSE', fontweight='bold')
ax2.legend()

# 3. Scatter plot: Actual vs Predicted (pro nejlepší target)
best_idx = results_df['R2'].idxmax()
best_target = results_df.loc[best_idx, 'Target']
ax3 = axes[1, 0]
ax3.scatter(y_test[:, best_idx], y_pred[:, best_idx], alpha=0.5, s=20)
ax3.plot([y_test[:, best_idx].min(), y_test[:, best_idx].max()],
         [y_test[:, best_idx].min(), y_test[:, best_idx].max()],
         'r--', linewidth=2, label='Ideální')
ax3.set_xlabel('Skutečná hodnota')
ax3.set_ylabel('Predikovaná hodnota')
ax3.set_title(f'Actual vs Predicted: {best_target} (R²={results_df.loc[best_idx, "R2"]:.3f})', fontweight='bold')
ax3.legend()

# 4. Distribuce R²
ax4 = axes[1, 1]
ax4.hist(results_df['R2'], bins=10, color='steelblue', edgecolor='black', alpha=0.7)
ax4.axvline(results_df['R2'].mean(), color='red', linestyle='--', 
            label=f'Průměr: {results_df["R2"].mean():.3f}')
ax4.set_xlabel('R² Score')
ax4.set_ylabel('Počet targetů')
ax4.set_title('Distribuce R² Score', fontweight='bold')
ax4.legend()

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/fundamental_predictor_evaluation.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {DATA_PATH}/fundamental_predictor_evaluation.png")

---
## 8. Feature Importance

Které OHLCV features jsou nejdůležitější pro predikci fundamentů?

In [ ]:
# Agregovaná feature importance přes všechny targets
importances = []

for estimator in model.estimators_:
    importances.append(estimator.feature_importances_)

# Průměr přes všechny targets
avg_importance = np.mean(importances, axis=0)

# DataFrame
importance_df = pd.DataFrame({
    'Feature': available_features,
    'Importance': avg_importance
}).sort_values('Importance', ascending=False)

print("📊 FEATURE IMPORTANCE (průměr přes všechny targets)")
print("="*50)
for _, row in importance_df.iterrows():
    bar = '█' * int(row['Importance'] * 50)
    print(f"{row['Feature']:<20} {row['Importance']:.3f} {bar}")

In [ ]:
# Vizualizace feature importance
fig, ax = plt.subplots(figsize=(10, 8))

colors = plt.cm.viridis(np.linspace(0, 0.8, len(importance_df)))
bars = ax.barh(importance_df['Feature'], importance_df['Importance'], color=colors)

ax.set_xlabel('Importance')
ax.set_title('Feature Importance pro Predikci Fundamentů', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Anotace
for bar, val in zip(bars, importance_df['Importance']):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{val:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/feature_importance.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {DATA_PATH}/feature_importance.png")

---
## 9. Uložení Modelu

In [ ]:
# Uložení modelu a scaleru
model_path = f"{MODEL_PATH}/fundamental_predictor.pkl"
scaler_path = f"{MODEL_PATH}/feature_scaler.pkl"

joblib.dump(model, model_path)
joblib.dump(scaler, scaler_path)

print(f"💾 Model uložen: {model_path}")
print(f"💾 Scaler uložen: {scaler_path}")

In [ ]:
# Uložení metadat modelu
metadata = {
    'features': available_features,
    'targets': available_targets,
    'rf_params': RF_PARAMS,
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'avg_r2': results_df['R2'].mean(),
    'avg_mae': results_df['MAE'].mean(),
    'created': datetime.now().isoformat()
}

# Uložení jako JSON
import json
metadata_path = f"{MODEL_PATH}/fundamental_predictor_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"💾 Metadata uložena: {metadata_path}")

In [ ]:
# Uložení výsledků evaluace
results_path = f"{DATA_PATH}/fundamental_predictor_results.csv"
results_df.to_csv(results_path, index=False)

print(f"💾 Výsledky uloženy: {results_path}")

---
## 10. Shrnutí a Další Kroky

### ✅ Dokončeno:

| Úkol | Status |
|------|--------|
| Příprava trénovacích dat | ✅ |
| Trénování Multi-Output RF | ✅ |
| Evaluace (MAE, RMSE, R²) | ✅ |
| Feature Importance analýza | ✅ |
| Uložení modelu | ✅ |

### 📁 Vytvořené soubory:

| Soubor | Popis |
|--------|-------|
| `models/fundamental_predictor.pkl` | Natrénovaný RF model |
| `models/feature_scaler.pkl` | StandardScaler pro features |
| `models/fundamental_predictor_metadata.json` | Metadata modelu |
| `data/fundamental_predictor_results.csv` | Evaluační metriky |

### ➡️ Další notebook:

**Notebook 03: Doplnění Historických Dat**
- Použití natrénovaného modelu pro imputaci 2015-2024
- Validace predikovaných hodnot
- Vytvoření kompletního 10-letého datasetu

In [ ]:
# Finální shrnutí
print("="*70)
print("📊 NOTEBOOK 02 - SHRNUTÍ")
print("="*70)

print(f"\n🌲 Model: Multi-Output Random Forest")
print(f"   • Stromy: {RF_PARAMS['n_estimators']}")
print(f"   • Hloubka: {RF_PARAMS['max_depth']}")

print(f"\n📊 Výsledky:")
print(f"   • Průměrné R²: {results_df['R2'].mean():.3f}")
print(f"   • Průměrné MAE: {results_df['MAE'].mean():.3f}")
print(f"   • Nejlepší target: {best_target} (R²={results_df.loc[best_idx, 'R2']:.3f})")

print(f"\n🔝 Top 3 nejdůležitější features:")
for i, (_, row) in enumerate(importance_df.head(3).iterrows()):
    print(f"   {i+1}. {row['Feature']}: {row['Importance']:.3f}")

print(f"\n✅ Model připraven pro Notebook 03!")